## Problem

1. Using Gaussian elimination to achieve the LU decomposition with and without a column pivoting; 

2. Using the two LU decomposition algorithm to solve linear systems in which the coefficient matrix is (1) general nonsingular matrix; (2) positive definite matrix; (3) diagonally dominant matrix.  
   
3. Compare the numerical accuracy for the two algorithms.

In [47]:
import numpy as np

def pair_pmat(n, ind1, ind2):
    """
    Calculate the pair permutation matrix which switch the column ind1 and ind2
    params:
        n: the dimension of mat
        ind1, ind2: the index to switch
    return:
        P: the permutation mat
    """
    P = np.eye(n)
    P[ind1, ind1] = 0; P[ind1, ind2] = 1
    P[ind2, ind2] = 0; P[ind2, ind1] = 1
    return P

def LU_pivot(mat:np.matrix) -> tuple[np.matrix, np.matrix, np.matrix]:
    """
    LU factorization by Guassian elimination with column pivoting
    param:
        mat: the matrix that needs LU factorization
    return:
        lower-diag matrix, upper-diag matrix, permutation matrix
    """
    n = np.size(mat, 0)
    L = np.eye(n); U = mat.copy(); P = np.eye(n)
    for k in range(n-1): # 每一行的计算步骤
        pivot_ind = np.argmax(np.abs(U[k,:])) # 主元位置
        pivot = U[k, pivot_ind]; P_tmp = pair_pmat(n, k, pivot_ind)
        P = P @ P_tmp; U = U @ P_tmp
        for kp in range(k+1,n):
            L[kp, k] = U[kp, k] / U[k, k] # 更新 L 第一列
            U[kp,:] = U[kp,:] - L[kp,k] * U[k,:] # 更新 U 第 2-n 列
    return L, U, P.T

def LU(mat:np.matrix) -> tuple[np.matrix, np.matrix]:
    """
    LU factorization by Guassian elimination without column pivoting
    param:
        mat: the matrix that needs LU factorization
    return:
        lower-diag matrix, upper-diag matrix
    """
    n = np.size(mat, 0)
    L = np.eye(n); U = mat.copy()
    for k in range(n-1): # 每一行的计算步骤
        for kp in range(k+1,n):
            L[kp, k] = U[kp, k] / U[k, k] # 更新 L 第一列
            U[kp,:] = U[kp,:] - L[kp,k] * U[k,:] # 更新 U 第 2-n 列
    return L, U
        
def LU_accuracy(mat, *factorization):
    """
    Calculate the accuracy of LU decomposition
    params:
        mat: the matrix that will be decomposited
        factorization: the factor matries of mat
    return:
        the inf norm of (mat - @ factorization)
    """
    factor_mul = np.linalg.multi_dot(*factorization)
    accu = np.linalg.norm(mat - factor_mul, ord = np.inf)
    return accu

以上是解决本次问题需要的函数，下面将详细解释每个函数的作用：
1. pair_pmat 函数用于创建一个仅仅执行一次对换的置换矩阵，也就是产生对换第 i 行和第 j 行的置换矩阵
2. LU_pivot 采用的算法是主元 LU 分解，需要多次置换矩阵参与
3. LU 采用的是非主元 LU 分解
4. LU_accuracy 是用于检测 LU 矩阵分解的精确度的，精确度定义如下：
   $$
   accu = \Vert A - LUP^T \Vert_\infty
   $$
   其中 A 是 LU 分解的对象，L,U,P 均是 LU 分解产生的下三角、上三角与置换矩阵

In [48]:
# general nonsingular matrix
np.random.seed(822)
A = np.random.randn(1000, 1000)
print("rank of A is",np.linalg.matrix_rank(A))
print("The accuracy of A\'s LU factorization without column pivot is", LU_accuracy(A, LU(A)))
print("The accuracy of A\'s LU factorization with column pivot is", LU_accuracy(A, LU_pivot(A)))

rank of A is 1000
The accuracy of A's LU factorization without column pivot is 6.7607794401909616e-09
The accuracy of A's LU factorization with column pivot is 7.996693964559126e-12


In [51]:
# positive definite matrix
np.random.seed(521)
B = np.random.randn(1000, 1000)
print("rank of B is",np.linalg.matrix_rank(B))
B = B.T @ B
print("The accuracy of B\'s LU factorization without column pivot is", LU_accuracy(B, LU(B)))
print("The accuracy of B\'s LU factorization with column pivot is", LU_accuracy(B, LU_pivot(B)))

rank of B is 1000
The accuracy of B's LU factorization without column pivot is 2.2741015667793185e-11
The accuracy of B's LU factorization with column pivot is 2.2741015667793185e-11


In [50]:
# diagonally dominant matrix
np.random.seed(515)
C = np.random.randn(1000, 1000)
C = C + np.diag(np.linalg.norm(C, axis = 1, ord = 1))*2
print("rank of C is",np.linalg.matrix_rank(C))
print("The accuracy of C\'s LU factorization without column pivot is", LU_accuracy(C, LU(C)))
print("The accuracy of C\'s LU factorization with column pivot is", LU_accuracy(C, LU_pivot(C)))

rank of C is 1000
The accuracy of C's LU factorization without column pivot is 6.578126064693546e-12
The accuracy of C's LU factorization with column pivot is 6.578126064693546e-12


结果明显可见：对于完全随机产生的矩阵 A, LU_pivot 的精度更高；但是对于本身就是正定矩阵或者对角优势的矩阵， LU_pivot 和 LU 的效果是相同的。